# Review Classification
#### In this project I am going to classify reviews in to positive and negetive reviews. I am going to use Embedding, LSTM and a fully connected layer at the end. For sake of learning I am not going to use any pretrained model. 

##### I am running on RTX 4070 gpu. At first I check how many unique words are in my dataset so I choose between word level and char level. I could use pretrained ones but I want to do it all by myself.  

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\panah\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\panah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
file_path = 'Datasets/dataset.csv'

df = pd.read_csv(file_path)

print(df.head())

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [4]:
all_reviews = ' '.join(df['review'].astype(str).tolist())

tokens = word_tokenize(all_reviews.lower()) 

translator = str.maketrans('', '', string.punctuation)

tokens = [word.translate(translator) for word in tokens]

stop_words = set(stopwords.words('english'))

filtered_words = [word for word in tokens if word.isalpha() and word not in stop_words]

unique_words = set(filtered_words)

num_unique_words = len(unique_words)

print(f"Number of unique words: {num_unique_words}")

Number of unique words: 133264


### Status: In progress